In [1]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4,8),
                    nn.ReLU(),
                    nn.Linear(8,1))
X = torch.rand((2,4))
net(X)

tensor([[-0.1510],
        [-0.1857]], grad_fn=<AddmmBackward0>)

我们从已有模型中访问参数。 当通过Sequential类定义模型时， 我们可以通过索引来访问模型的任意层。 这就像模型是一个列表一样，每层的参数都在其属性中。 如下所示，我们可以检查第二个全连接层的参数。

In [2]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[ 0.0486, -0.3085, -0.2250,  0.3045,  0.3234, -0.2745,  0.2093,  0.3243]])), ('bias', tensor([0.1060]))])


In [3]:
print(net[2].bias)
print(type(net[2].bias))
print(net[2].bias.data)

Parameter containing:
tensor([0.1060], requires_grad=True)
<class 'torch.nn.parameter.Parameter'>
tensor([0.1060])


In [5]:
net[0].weight.grad == None

True

In [10]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [11]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [14]:
def block1():
    return nn.Sequential(nn.Linear(4,8),
                         nn.ReLU(),
                         nn.Linear(8,4),
                         nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4,1))
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [18]:
print(rgnet[0][1][0].weight.data)
print(rgnet[0][1][0].bias.data)

tensor([[ 0.2804, -0.2160, -0.4282,  0.4410],
        [ 0.0186,  0.3117,  0.2166, -0.0310],
        [-0.1801,  0.2771,  0.2055,  0.3679],
        [ 0.4362, -0.3521, -0.1982,  0.4132],
        [-0.2567,  0.4600, -0.3274, -0.4863],
        [ 0.4787,  0.3080, -0.1252,  0.1397],
        [ 0.2691,  0.2021, -0.3579, -0.4939],
        [ 0.4806, -0.1652, -0.4275, -0.2790]])
tensor([-0.4174, -0.0306, -0.4539,  0.4606, -0.0393,  0.2274,  0.2893, -0.3195])


In [22]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
print(net)
net[0].weight.data[0], net[0].bias.data[0]

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


(tensor([-0.0002, -0.0119,  0.0012,  0.0005]), tensor(0.))

In [25]:
# 参数初始化为给定的常数
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight

Parameter containing:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], requires_grad=True)

In [26]:
# 参数绑定
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8,1))

net(X)
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])


In [28]:
# 他们是同一个对象，而不只是数值相同
net[2].weight.data[0,0]=1314
print(net[2].weight.data[0,0])
print(net[2].weight.data[0]==net[4].weight.data[0])

tensor(1314.)
tensor([True, True, True, True, True, True, True, True])
